<a href="https://colab.research.google.com/github/rushikeshnaik779/classification_algorithms_miscellaneous/blob/main/D2V_W2V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import nltk 
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#downloding data
!wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
!wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
!ls -lah DATAPATH

--2021-02-04 10:29:15--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2479133 (2.4M) [text/plain]
Saving to: ‘DATAPATH/train_data.csv’

train_data.csv      100%[===================>]   2.36M  --.-KB/s    in 0.1s    

2021-02-04 10:29:15 (20.0 MB/s) - ‘DATAPATH/train_data.csv’ saved [2479133/2479133]

--2021-02-04 10:29:15--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.

In [3]:
# load the dataset and explore 
filepath = "DATAPATH/train_data.csv"

df = pd.read_csv(filepath)
print(df.shape)
df.head()

(30000, 2)


,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
df['sentiment'].value_counts()

worry         7433
neutral       6340
sadness       4828
happiness     2986
love          2068
surprise      1613
hate          1187
fun           1088
relief        1021
empty          659
enthusiasm     522
boredom        157
anger           98
Name: sentiment, dtype: int64

In [5]:
# let us take the top 3 categories and leave out the rest 
shortlist = ['neutral', 'happiness', 'worry']
df_subset = df[df['sentiment'].isin(shortlist)]
df_subset.shape

(16759, 2)

## Text pre-processing:

Tweets are different. Somethings to consider:
- Removing @mentions and urls
- using NLTK Tweet tokenizer instead of a regular one 
- stopwords, numbers as usual.

In [6]:
tweeter = TweetTokenizer(strip_handles=True, preserve_case=False)
mystopwords = set(stopwords.words("english"))


def preprocess_corpus(texts):
    def remove_stops_digit(tokens):
        return [token for token in tokens if token not in mystopwords and not token.isdigit()]
    
    return [remove_stops_digit(tweeter.tokenize(content)) for content in texts]

In [7]:
mydata = preprocess_corpus(df_subset['content'])
mycats = df_subset['sentiment']
print(len(mydata), len(mycats))

16759 16759


In [8]:
# spliting the data into train and test, following the usual process 
X_train , X_test, y_train, y_test = train_test_split(mydata, mycats, random_state=100)

train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(X_train)]
model = Doc2Vec(vector_size=50, alpha=0.025, min_count=5, dm=1, epochs=100)
model.build_vocab(train_doc2vec)
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)
model.save("d2v.model")
print("Model saved")

Model saved


In [9]:
model = Doc2Vec.load('d2v.model')

train_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in X_train]
test_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in X_test]


In [10]:
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB

myclass = LogisticRegression(class_weight="balanced")
myclass.fit(train_vectors, y_train)

preds = myclass.predict(test_vectors)

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

   happiness       0.37      0.53      0.43       741
     neutral       0.47      0.54      0.50      1567
       worry       0.59      0.41      0.49      1882

    accuracy                           0.48      4190
   macro avg       0.48      0.50      0.48      4190
weighted avg       0.51      0.48      0.48      4190



WORD2VEC


In [12]:
!wget -P DATAPATH -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-02-04 10:32:41--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.71.78
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.71.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘DATAPATH/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  44.9MB/s    in 35s     

2021-02-04 10:33:16 (45.2 MB/s) - ‘DATAPATH/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [13]:
import os 
from time import time 

# preprocessing imports
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation


# imports related to modeling 
import numpy as np 
from gensim.models import Word2Vec, keyedvectors, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [14]:
! cat DATAPATH/amazon_cells_labelled.txt DATAPATH/imdb_labelled.txt DATAPATH/yelp_labelled.txt > sentiment_sentences.txt

In [15]:
data_path = "DATAPATH"
path_to_model = os.path.join(data_path, 'GoogleNews-vectors-negative300.bin.gz')


In [16]:
# load w2v model this will take some time 
%time w2v_model = KeyedVectors.load_word2vec_format(path_to_model, binary=True)
print("done loading word2vec")

CPU times: user 1min 45s, sys: 2.03 s, total: 1min 47s
Wall time: 1min 47s
done loading word2vec


In [42]:
texts =[]
cats = []
fh = open("sentiment_sentences.txt")
for line in fh : 
    text, sentiment = line.split("\t")
    texts.append(text)
    cats.append(sentiment)


In [19]:
word2vec_vocab = w2v_model.vocab.keys()
word2vec_vocab_lower = [item.lower() for item in word2vec_vocab]
print(len(word2vec_vocab))

3000000


In [20]:
print(len(cats), len(texts))
print(texts[1])
print(cats[1])

3000 3000
Good case, Excellent value.
1



In [31]:
# preprocess the text 
nltk.download('punkt')
def preprocess_corpus(texts):
    mystopwords = set(stopwords.words("english"))

    def remove_stops_digit(tokens):
        return [token.lower() for token in tokens if token not in mystopwords 
                and not token.isdigit() and token not in punctuation]

    return [remove_stops_digit(word_tokenize(text)) for text in texts]


texts_processed = preprocess_corpus(texts)
print(len(cats), len(texts_processed))
print(texts_processed[1])
print(cats[1])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
3000 3000
['good', 'case', 'excellent', 'value']
1



In [27]:
# creating a feature vector by averaging all embeddings for all sentences 
def embedding_feats(list_of_lists):
    DIMENSION = 300
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists: 
        feat_for_this = np.zeros(DIMENSION)
        count_for_this = 0 

        for token in tokens: 
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this += 1
        feats.append(feat_for_this/count_for_this)
    
    return feats

In [28]:
train_vectors = embedding_feats(texts_processed)
print(len(train_vectors))

3000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


ValueError: ignored

In [43]:
train_vectors = train_vectors.fillna(train_vectors.mean())

AttributeError: ignored

In [45]:
train_vectors = pd.DataFrame(train_vectors)

In [46]:
train_vectors['cats'] = cats

In [47]:
train_vectors = train_vectors.fillna(train_vectors.mean())



In [48]:
# taking logistic regression 
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
classifier = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(train_vectors.drop(['cats'], axis=1), train_vectors['cats'])
classifier.fit(X_train, y_train)
print("Accuracy", classifier.score(X_test, y_test))

Accuracy 0.8386666666666667


In [51]:
preds = classifier.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          0
       0.84      0.85      0.85       391
          1
       0.83      0.83      0.83       359

    accuracy                           0.84       750
   macro avg       0.84      0.84      0.84       750
weighted avg       0.84      0.84      0.84       750

